In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

In [2]:
images = glob.glob('camera_cal/calibration*.jpg')

objpoints = []
imgpoints = []

objp = np.zeros((9*6, 3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

for image in images:
    img = mpimg.imread(image)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

In [3]:
img = cv2.imread('./camera_cal/calibration2.jpg')
img_size = (img.shape[1], img.shape[0])

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

In [4]:
def warp(img):  
    img_size = (img.shape[1], img.shape[0])
    height, width = img.shape[:2]
    src = np.float32([[600,480],[720,480],[280,660],[1030,660]])
    dst = np.float32([[450,0],[width-450,0],[450,height],[width-450,height]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped

def unwarp(img):  
    img_size = (img.shape[1], img.shape[0])
    height, width = img.shape[:2]
    src = np.float32([[600,480],[720,480],[280,660],[1030,660]])
    dst = np.float32([[450,0],[width-450,0],[450,height],[width-450,height]])

    Minv = cv2.getPerspectiveTransform(dst, src)
    
    unwarped = cv2.warpPerspective(img, Minv, img_size)
    
    return unwarped

In [5]:
def binary(img, thresh=(230,255)):
    if (np.max(img) > 170):
        img = img*(255/np.max(img))
    
    binary_output = np.zeros_like(img)
    binary_output[((img >= thresh[0]) & (img <= thresh[1]))] = 1
    
    return binary_output

def lines(img):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    warped = warp(undist)

    warped_HSV = cv2.cvtColor(warped, cv2.COLOR_RGB2HSV)
    warped_LAB = cv2.cvtColor(warped, cv2.COLOR_RGB2LAB)

    warped_R = warped[:,:,0]
    warped_R = binary(warped_R)
    
    warped_B = warped_LAB[:,:,2]
    warped_B = binary(warped_B, thresh=(210,255))

    combined = np.zeros_like(warped_R)
    combined[(warped_R == 1) | (warped_B == 1)] = 1
    
    return combined

In [6]:
def sliding_windows(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    left_offset = np.int(histogram.shape[0] * 0.08)
    right_offset = np.int(histogram.shape[0] * 0.92)
    
    leftx_base = np.argmax(histogram[left_offset:midpoint]) + left_offset
    rightx_base = np.argmax(histogram[midpoint:right_offset]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = None
    right_fit = None
    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds

In [7]:
def region(binary_warped, left_fit, right_fit):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = None
    right_fit = None
    if len(leftx) != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) != 0:
        right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds

In [21]:
def measure_curvature(binary_warped, left_lane_inds, right_lane_inds, left_fit, right_fit):
    left_curverad = 0
    right_curverad = 0
    center = 0
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    h,w = binary_warped.shape
    ploty = np.linspace(0, h-1, h)
    y_eval = np.max(ploty)
    # Fit new polynomials to x,y in world space
    if len(leftx) != 0 and len(rightx) != 0:
        left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

        # Calculate the new radii of curvature
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    if left_fit is not None and right_fit is not None:
        # Now our radius of curvature is in meters
        left_fit_bottom = left_fit[0]*h**2 + left_fit[1]*h + left_fit[2]
        right_fit_bottom = right_fit[0]*h**2 + right_fit[1]*h + right_fit[2]
        car_pos = w/2
        center = (car_pos - (left_fit_bottom + right_fit_bottom) / 2) * xm_per_pix
    
    return left_curverad, right_curverad, center

In [9]:
def draw(img, left_fitx, right_fitx, left_curverad, right_curverad, center):
    new_warped = np.zeros_like(img).astype(np.uint8)
    
    ploty = np.linspace(0, new_warped.shape[0]-1, new_warped.shape[0] )
    left_points = np.array([np.transpose(np.vstack((left_fitx, ploty)))], np.int32)
    right_points = np.array([np.flipud(np.transpose(np.vstack((right_fitx, ploty))))], np.int32)
    points = np.hstack((left_points, right_points))
    
    cv2.fillPoly(new_warped, points, (0,255,0))
    cv2.polylines(new_warped, left_points, isClosed=False, color=(255,0,0), thickness=20)
    cv2.polylines(new_warped, right_points, isClosed=False, color=(0,0,255), thickness=20)
    
    new_unwarped = unwarp(new_warped)
    combined = cv2.addWeighted(img, 1, new_unwarped, 1, 0)
    
    font = cv2.FONT_HERSHEY_DUPLEX
    text = 'Left curve radius: ' + '{:04.2f}'.format(left_curverad) + 'm'
    cv2.putText(combined, text, (40,50), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
    text = 'Right curve radius: ' + '{:04.2f}'.format(right_curverad) + 'm'
    cv2.putText(combined, text, (40,100), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
    text = 'The position of vehicle:' + '{:04.3f}'.format(center) + 'm '
    cv2.putText(combined, text, (40,150), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
    
    return combined

In [10]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = []  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    def modify(self, fit, inds):
        n = 5
        if fit is not None:
            if self.best_fit is not None:
                self.diffs = abs(fit - self.best_fit)
            if (self.diffs[0] > 0.001 or self.diffs[1] > 1. or self.diffs[2] > 100.) and len(self.current_fit) != 0:
                self.detected = False
            else:
                if len(self.current_fit) < n:
                    self.current_fit.append(fit)
                else:
                    self.current_fit.pop(0)
                    self.current_fit.append(fit)
                self.detected = True
                self.best_fit = np.average(self.current_fit, axis=0)
        else:
            self.detected = False

In [15]:
def process_image(img):
    binary_warped = lines(img)

    if not left.detected or not right.detected:
        left_fit, right_fit, left_lane_inds, right_lane_inds = sliding_windows(binary_warped)
    else:
        left_fit, right_fit, left_lane_inds, right_lane_inds = region(binary_warped, left.best_fit, right.best_fit)

    left.modify(left_fit, left_lane_inds)
    right.modify(right_fit, right_lane_inds)
    
    left_curverad, right_curverad, center = measure_curvature(binary_warped, left_lane_inds, right_lane_inds, left_fit, right_fit)

    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left.best_fit[0]*ploty**2 + left.best_fit[1]*ploty + left.best_fit[2]
    right_fitx = right.best_fit[0]*ploty**2 + right.best_fit[1]*ploty + right.best_fit[2]
    
    combined = draw(img, left_fitx, right_fitx, left_curverad, right_curverad, center)

    return combined

In [12]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [13]:
left = Line()
right = Line()
white_output = 'project_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_video_out.mp4
[MoviePy] Writing video project_video_out.mp4


100%|█████████▉| 1260/1261 [03:23<00:00,  6.18it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_out.mp4 

CPU times: user 3min 52s, sys: 19.2 s, total: 4min 11s
Wall time: 3min 25s


In [22]:
left = Line()
right = Line()
white_output2 = 'challenge_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip2 = VideoFileClip("challenge_video.mp4")
white_clip2 = clip2.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip2.write_videofile(white_output2, audio=False)

[MoviePy] >>>> Building video challenge_video_out.mp4
[MoviePy] Writing video challenge_video_out.mp4






  0%|          | 0/485 [00:00<?, ?it/s]



  0%|          | 1/485 [00:00<01:33,  5.19it/s]



  0%|          | 2/485 [00:00<01:35,  5.04it/s]



  1%|          | 3/485 [00:00<01:34,  5.08it/s]



  1%|          | 4/485 [00:00<01:33,  5.16it/s]



  1%|          | 5/485 [00:00<01:30,  5.29it/s]



  1%|          | 6/485 [00:01<01:29,  5.34it/s]



  1%|▏         | 7/485 [00:01<01:30,  5.29it/s]



  2%|▏         | 8/485 [00:01<01:31,  5.23it/s]



  2%|▏         | 9/485 [00:01<01:31,  5.19it/s]



  2%|▏         | 10/485 [00:01<01:31,  5.21it/s]



  2%|▏         | 11/485 [00:02<01:30,  5.23it/s]



  2%|▏         | 12/485 [00:02<01:30,  5.23it/s]



  3%|▎         | 13/485 [00:02<01:30,  5.22it/s]



  3%|▎         | 14/485 [00:02<01:30,  5.21it/s]



  3%|▎         | 15/485 [00:02<01:28,  5.33it/s]



  3%|▎         | 16/485 [00:02<01:26,  5.44it/s]



  4%|▎         | 17/485 [00:03<01:24,  5.56it/s]



  4%|▎         | 18/485 [00:03<01:22,  5.66it/s]



  4%|▍         | 19/485 [0

 26%|██▌       | 127/485 [00:19<00:56,  6.37it/s]



 26%|██▋       | 128/485 [00:20<00:55,  6.38it/s]



 27%|██▋       | 129/485 [00:20<00:55,  6.39it/s]



 27%|██▋       | 130/485 [00:20<00:55,  6.40it/s]



 27%|██▋       | 131/485 [00:20<00:55,  6.41it/s]



 27%|██▋       | 132/485 [00:20<00:54,  6.42it/s]



 27%|██▋       | 133/485 [00:20<00:54,  6.43it/s]



 28%|██▊       | 134/485 [00:20<00:54,  6.44it/s]



 28%|██▊       | 135/485 [00:20<00:54,  6.45it/s]



 28%|██▊       | 136/485 [00:21<00:53,  6.46it/s]



 28%|██▊       | 137/485 [00:21<00:53,  6.48it/s]



 28%|██▊       | 138/485 [00:21<00:53,  6.48it/s]



 29%|██▊       | 139/485 [00:21<00:53,  6.49it/s]



 29%|██▉       | 140/485 [00:21<00:53,  6.49it/s]



 29%|██▉       | 141/485 [00:21<00:53,  6.48it/s]



 29%|██▉       | 142/485 [00:21<00:52,  6.48it/s]



 29%|██▉       | 143/485 [00:22<00:52,  6.47it/s]



 30%|██▉       | 144/485 [00:22<00:52,  6.47it/s]



 30%|██▉       | 145/485 [00:22<00:52,  6.47it

 52%|█████▏    | 253/485 [00:39<00:36,  6.40it/s]



 52%|█████▏    | 254/485 [00:39<00:36,  6.39it/s]



 53%|█████▎    | 255/485 [00:39<00:35,  6.40it/s]



 53%|█████▎    | 256/485 [00:39<00:35,  6.40it/s]



 53%|█████▎    | 257/485 [00:40<00:35,  6.40it/s]



 53%|█████▎    | 258/485 [00:40<00:35,  6.39it/s]



 53%|█████▎    | 259/485 [00:40<00:35,  6.38it/s]



 54%|█████▎    | 260/485 [00:40<00:35,  6.38it/s]



 54%|█████▍    | 261/485 [00:40<00:35,  6.37it/s]



 54%|█████▍    | 262/485 [00:41<00:35,  6.36it/s]



 54%|█████▍    | 263/485 [00:41<00:34,  6.36it/s]



 54%|█████▍    | 264/485 [00:41<00:34,  6.35it/s]



 55%|█████▍    | 265/485 [00:41<00:34,  6.34it/s]



 55%|█████▍    | 266/485 [00:41<00:34,  6.34it/s]



 55%|█████▌    | 267/485 [00:42<00:34,  6.34it/s]



 55%|█████▌    | 268/485 [00:42<00:34,  6.33it/s]



 55%|█████▌    | 269/485 [00:42<00:34,  6.33it/s]



 56%|█████▌    | 270/485 [00:42<00:34,  6.31it/s]



 56%|█████▌    | 271/485 [00:42<00:33,  6.31it

 78%|███████▊  | 379/485 [01:01<00:17,  6.19it/s]



 78%|███████▊  | 380/485 [01:01<00:16,  6.19it/s]



 79%|███████▊  | 381/485 [01:01<00:16,  6.19it/s]



 79%|███████▉  | 382/485 [01:01<00:16,  6.19it/s]



 79%|███████▉  | 383/485 [01:01<00:16,  6.19it/s]



 79%|███████▉  | 384/485 [01:02<00:16,  6.19it/s]



 79%|███████▉  | 385/485 [01:02<00:16,  6.19it/s]



 80%|███████▉  | 386/485 [01:02<00:15,  6.19it/s]



 80%|███████▉  | 387/485 [01:02<00:15,  6.19it/s]



 80%|████████  | 388/485 [01:02<00:15,  6.19it/s]



 80%|████████  | 389/485 [01:02<00:15,  6.18it/s]



 80%|████████  | 390/485 [01:03<00:15,  6.18it/s]



 81%|████████  | 391/485 [01:03<00:15,  6.18it/s]



 81%|████████  | 392/485 [01:03<00:15,  6.17it/s]



 81%|████████  | 393/485 [01:03<00:14,  6.17it/s]



 81%|████████  | 394/485 [01:03<00:14,  6.17it/s]



 81%|████████▏ | 395/485 [01:04<00:14,  6.17it/s]



 82%|████████▏ | 396/485 [01:04<00:14,  6.17it/s]



 82%|████████▏ | 397/485 [01:04<00:14,  6.17it

[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_video_out.mp4 

CPU times: user 1min 32s, sys: 7.71 s, total: 1min 40s
Wall time: 1min 20s


In [81]:
left = Line()
right = Line()
white_output3 = 'harder_challenge_video_out.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip3 = VideoFileClip("harder_challenge_video.mp4")
white_clip3 = clip3.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip3.write_videofile(white_output3, audio=False)

[MoviePy] >>>> Building video harder_challenge_video_out.mp4
[MoviePy] Writing video harder_challenge_video_out.mp4



 25%|██▌       | 303/1200 [00:45<02:13,  6.71it/s]


 50%|█████     | 605/1200 [01:33<01:31,  6.47it/s]


 76%|███████▌  | 907/1200 [02:24<00:46,  6.29it/s]


100%|█████████▉| 1199/1200 [03:04<00:00,  6.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: harder_challenge_video_out.mp4 

CPU times: user 3min 29s, sys: 20.4 s, total: 3min 50s
Wall time: 3min 6s
